In [ ]:
#5장의 컨브넷 필터 시각화 기법과 거의 동일(상위 층에 있는 특정 필터의 활성화를 극대화하기 위해 컨브넷의 입력 경사 상승법을 적용)
*특정 필터가 아니라 전체 층의 활성화를 최대화, 한꺼번에 많은 특성을 섞어 시각화
*빈 이미지나 노이즈가 조금 있는 입력이 아니라 이미 가지고 있는 이미지를 사용
*입력 이미지는 시각 품질을 높이기 위해 여러 다른 스케일(옥타브(octave))로 처리한다.

옥타브 : 이미지 크기를 일정한 비율로 연속적으로 줄이거나 늘리는 방식

In [2]:
import os
currentPath = os.getcwd()
print(currentPath)
# os.chdir('/Users/jaekunpark/jupyter_project')

/home/hongbeom/Workspace/JKP


In [3]:
import tensorflow as tf
import numpy as np
import scipy


from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.applications import inception_v3
from tensorflow.keras.preprocessing import image

tf.compat.v1.disable_eager_execution()

K.set_learning_phase(0) # Sets the learning phase to a fixed value. # 여기서는 모델 훈련 X, 모든 훈련 연산 비활성화

model = inception_v3.InceptionV3(weights = 'imagenet', include_top = False)

# 레이어 가중치 설정
layer_contributions = {'mixed2':0.2, 'mixed3':0.3, 'mixed4':2., 'mixed5':1.5,}
    # 층 이름과 계수를 매핑한 딕셔너리, 최대화하려는 손실에 층의 활성화가 기여할 양을 선택

# 최대화할 손실 정의
layer_dict = dict([(layer.name, layer) for layer in model.layers])
loss = K.variable(0.)    # Instantiates a variable and returns it.
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output # 층의 출력을 얻는다.
    
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss = loss + coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling
    # 층 특성의 L2 Norm 제곱을 손실에 추가(이미지 테두리 제외)
    # 층마다 활성화의 크기가 다르기 떄문에 정규화를 위해 전체 원소의 개수를 계산
    
# prod : Multiplies the values in a tensor, alongside the specified axis.   
# cast : Casts a tensor to a different dtype and returns it.
# shape : Returns the symbolic shape of a tensor or variable.

# model.summary()




# Gradient Ascending Process

dream = model.input # 이 텐서는 생성된 이미지 저장
grads = K.gradients(loss, dream)[0] # Returns the gradients of loss w.r.t. variables. (손실에 대한 이미지 그래디언트 계산)
grads = grads / K.maximum(K.mean(K.abs(grads)), 1e-7) # gradient normalization
    # maximum : Element-wise maximum of two tensors.

outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)
    # 주어진 입력 이미지에서 손실과 그래디언트 값을 계산할 케라스 Function 객체를 만든다.
    # Instantiates a variable and returns it. (input, output)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss = None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...', i, '번째 손실:', loss_value)
        x += step * grad_values
    return x

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
87916544/87910968 [==============================] - 104s 1us/step


In [4]:
# Utility Function

def resize_img(img, size):
    img = np.copy(img)
    factors = (1, float(size[0]) / img.shape[1], float(size[1]) / img.shape[2], 1)
    return scipy.ndimage.zoom(img, factors, order = 1)

def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    image.save_img(fname, pil_img)
    
def preprocess_img(image_path): # 사진을 열고 크기를 줄인 뒤, 인셉션 V3가 인식하는 텐서 포맷으로 변환
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    img = inception_v3.preprocess_input(img)
    return img

def deprocess_image(x): # 넘파이 배열을 적절한 이미지 포맷으로 변환하는 유틸리티 함수
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
        
    x /= 2.
    x += 0.5
    x *= 255. 
    x = np.clip(x, 0, 255).astype('uint8')
    return x

    # inception_v3.preprocess_input 함수에서 수행한 preprocess 과정을 복원 
    #(Xception, InceptionResnetV2, MobileNet, NASNet도 동일한 전처리)
    # 픽셀 값을 255로 나누어 0~1사이로 만든 후 -0.5, *2를 통해 -1~1사이의 값으로 만든다.
        
        

        
        
# Gradient Ascend in continuous scale

step = 0.01 
num_octave = 3
octave_scale = 1.4
iterations = 30

max_loss = 10. # 손실이 10보다 커지면 경사 상승법 중지

base_image_path = '/home/hongbeom/Workspace/JKP/Boyhood.jpg'

img = preprocess_img(base_image_path)

original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape) # 경사 상승법을 실행할 스케일 크기를 정의한 튜플의 리스트 준비

successive_shapes = successive_shapes[::-1] # 이 리스트를 크기순으로 뒤집는다.

original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('처리할 이미지 크기', shape)
    img = resize_img(img, shape) # scale up
    img = gradient_ascent(img, iterations = iterations, step = step, max_loss = max_loss)
    
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape) # 작게 줄인 원본 이미지의 스케일 높임
    same_size_original = resize_img(original_img, shape) # 이 크기에 해당하는 원본 이미지의 고해상도 버전 계산
    lost_detail = same_size_original - upscaled_shrunk_original_img
    
    img += lost_detail # 손실된 디테일을 딥드림 이미지에 다시 주입
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname = '/home/hongbeom/Workspace/JKP/dream_at_scale_' + str(shape) + '.png')
    
save_img(img, fname = '/home/hongbeom/Workspace/JKP/final_picture.png')

처리할 이미지 크기 (510, 510)
... 0 번째 손실: 0.58765143
... 1 번째 손실: 0.8211911
... 2 번째 손실: 1.1290452
... 3 번째 손실: 1.4748727
... 4 번째 손실: 1.8278415
... 5 번째 손실: 2.1839423
... 6 번째 손실: 2.5105157
... 7 번째 손실: 2.8496103
... 8 번째 손실: 3.1766157
... 9 번째 손실: 3.4859707
... 10 번째 손실: 3.7593288
... 11 번째 손실: 4.051634
... 12 번째 손실: 4.3335032
... 13 번째 손실: 4.607343
... 14 번째 손실: 4.8838506
... 15 번째 손실: 5.1382413
... 16 번째 손실: 5.3927493
... 17 번째 손실: 5.6502147
... 18 번째 손실: 5.9209375
... 19 번째 손실: 6.1693254
... 20 번째 손실: 6.4584503
... 21 번째 손실: 6.7511616
... 22 번째 손실: 7.06334
... 23 번째 손실: 7.432629
... 24 번째 손실: 7.842941
... 25 번째 손실: 8.369748
... 26 번째 손실: 8.863329
... 27 번째 손실: 9.631153
처리할 이미지 크기 (714, 714)
... 0 번째 손실: 1.829312
... 1 번째 손실: 3.6219301
... 2 번째 손실: 5.747384
... 3 번째 손실: 8.731686
처리할 이미지 크기 (1000, 1000)
... 0 번째 손실: 3.596242
... 1 번째 손실: 8.012013


In [1]:
import scipy
scipy.show_config()

lapack_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/home/hongbeom/.conda/envs/jktest3/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/home/hongbeom/.conda/envs/jktest3/include']
lapack_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/home/hongbeom/.conda/envs/jktest3/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/home/hongbeom/.conda/envs/jktest3/include']
blas_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/home/hongbeom/.conda/envs/jktest3/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/home/hongbeom/.conda/envs/jktest3/include']
blas_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/home/hongbeom/.conda/envs/jktest3/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/home/hongbeom/.conda/envs/jktest3/include']


In [ ]:
'''
먼저 이미지를 처리하기 위한 스케일(octave)리스트를 정의한다. 스케일은 이전 스케일 보다 1.4배 더 크다
가장 작은 것에서 가장 큰 스케일까지 연속적인 각 단계에서 정의한 손실이 최대화되도록 경사 상승법을 수행.
경사 상승법이 실행된 후 이미지 크기를 40%증가. 

예를 들어 350*350 크기의 원본 이미지를 178*178 크기로 줄여서 시작한다.
그 다음 40%씩 두 번 늘려서 178*178, 250*250, 350*350 크기에서 총 세 번 딥드립을 수행

스케일을 연속적으로 증가시키면서(점점 뭉개지거나 픽셀 경계가 나타나므로) 이미지 상세를 많이 잃지 않도록 간단한 기교를 사용
스케일을 늘린 후 이미지에 손실된 디테일을 재주입(원본 이미지가 크기를 늘렸을 때 어땠는지 알기 떄문에 가능)
작은 이미지 크기 S와 큰 이미지 크기 L이 주어지면 크기 L로 변경된 원본 이미지와 크기 S로 변경된 원본 이미지 사이의 차이를 계산
이 차이가 S에서 L로 변경되었을 때 잃어버린 디테일
'''